# OpenCV를 이용한 명함인식 구현 Tutorial - vol3
[Tutorial - vol02](https://github.com/ExcelsiorCJH/Projects/blob/master/Name%20Card%20Recognition/tutorial-vol2.ipynb)를 확장하여 노트북의 웹캠을 이용하여 실시간 명함인식기 구현 튜토리얼

In [1]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt

## 1. 웹캠을 이용해 Edge Detection 해보기

In [4]:
try: 
    cap = cv2.VideoCapture(0)
except:
    print ('cannot load camera!')

while True:
    ret, frame = cap.read()
    if not ret:
        print ('cannot load camera!')
        break

    k = cv2.waitKey(10)
    if k == 27:
        break

    # convert the image to grayscale, blur it, and find edges
    # in the image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    edged = cv2.Canny(gray, 75, 200)

    # show the original image and the edge detected image
    print ("STEP 1: Edge Detection")

    # find the contours in the edged image, keeping only the
    # largest ones, and initialize the screen contour
    (_, cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

    # loop over the contours
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        screenCnt = []

        # if our approximated contour has four points, then we
        # can assume that we have found our screen
        if len(approx) == 4:
            contourSize = cv2.contourArea(approx)
            camSize = frame.shape[0] * frame.shape[1]
            ratio = contourSize / camSize
            print (contourSize)
            print (camSize)
            print (ratio)

            if ratio > 0.1:
                screenCnt = approx

            break 

    if len(screenCnt) == 0:
        cv2.imshow("WebCam", frame)
        continue

    else:
        # show the contour (outline) of the piece of paper
        print ("STEP 2: Find contours of paper")

        cv2.drawContours(frame, [screenCnt], -1, (0, 255, 0), 2)
        cv2.imshow("WebCam", frame)


cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

STEP 1: Edge Detection
651.0
921600
0.0007063802083333334
STEP 1: Edge Detection
524.5
921600
0.0005691189236111111
STEP 1: Edge Detection
292.5
921600
0.0003173828125
STEP 1: Edge Detection
116.0
921600
0.00012586805555555555
STEP 1: Edge Detection
STEP 1: Edge Detection
STEP 1: Edge Detection
STEP 1: Edge Detection
98.5
921600
0.00010687934027777777
STEP 1: Edge Detection
292.5
921600
0.0003173828125
STEP 1: Edge Detection
1443.0
921600
0.0015657552083333333
STEP 1: Edge Detection
85.5
921600
9.27734375e-05
STEP 1: Edge Detection
84.0
921600
9.114583333333334e-05
STEP 1: Edge Detection
324.0
921600
0.0003515625
STEP 1: Edge Detection
33.5
921600
3.6349826388888886e-05
STEP 1: Edge Detection
STEP 1: Edge Detection
274.0
921600
0.0002973090277777778
STEP 1: Edge Detection
STEP 1: Edge Detection
274.0
921600
0.0002973090277777778
STEP 1: Edge Detection
368.0
921600
0.0003993055555555556
STEP 1: Edge Detection
500.5
921600
0.0005430772569444444
STEP 1: Edge Detection
440.0
921600
0.00047

-1

## 2. Perspective Transform 에서 부터 Threshold 처리까지

In [ ]:
def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
    rect = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    # return the ordered coordinates
    return rect

try:
    cap = cv2.VideoCapture(0)
except:
    print('cannot load camera!')

while True:
    ret, frame = cap.read()
    if not ret:
        print('cannot load camera!')
        break
    
    k = cv2.waitKey(10) & 0xff
    if k == 27:
        break
        
    # convert the image to grayscale, blur it, and find edges
    # in the image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)
    edged = cv2.Canny(gray, 75, 200)

    # show the original image and the edge detected image
    print ("STEP 1: Edge Detection")

    # find the contours in the edged image, keeping only the
    # largest ones, and initialize the screen contour
    (_, cnts, _) = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

    # loop over the contours
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
        screenCnt = []

        # if our approximated contour has four points, then we
        # can assume that we have found our screen
        if len(approx) == 4:
            contourSize = cv2.contourArea(approx)
            camSize = frame.shape[0] * frame.shape[1]
            ratio = contourSize / camSize
            print (contourSize)
            print (camSize)
            print (ratio)

            if ratio > 0.1:
                screenCnt = approx

            break 

    if len(screenCnt) == 0:
        cv2.imshow("WebCam", frame)
        continue

    else:
        # show the contour (outline) of the piece of paper
        print ("STEP 2: Find contours of paper")

        cv2.drawContours(frame, [screenCnt], -1, (0, 255, 0), 2)
        cv2.imshow("WebCam", frame)
        
        # apply the four point transform to obtain a top-down
        # view of the original image
        rect = order_points(screenCnt.reshape(4, 2))
        (topLeft, topRight, bottomRight, bottomLeft) = rect

        w1 = abs(bottomRight[0] - bottomLeft[0])
        w2 = abs(topRight[0] - topLeft[0])
        h1 = abs(topRight[1] - bottomRight[1])
        h2 = abs(topLeft[1] - bottomLeft[1])
        maxWidth = max([w1, w2])
        maxHeight = max([h1, h2])

        dst = np.float32([[0,0], [maxWidth-1,0], 
                          [maxWidth-1,maxHeight-1], [0,maxHeight-1]])

        M = cv2.getPerspectiveTransform(rect, dst)
        warped = cv2.warpPerspective(frame, M, (maxWidth, maxHeight))

        # show the original and scanned images
        print ("STEP 3: Apply perspective transform")

        # convert the warped image to grayscale, then threshold it
        # to give it that 'black and white' paper effect
        warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
        warped = cv2.adaptiveThreshold(warped, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 21, 10)

        # show the original and scanned images
        print ("STEP 4: Apply Adaptive Threshold")

        break


cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

cv2.imshow("Scanned", warped)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

STEP 1: Edge Detection
STEP 1: Edge Detection
STEP 1: Edge Detection
569.0
921600
0.0006174045138888889
STEP 1: Edge Detection
109.5
921600
0.00011881510416666667
STEP 1: Edge Detection
608.0
921600
0.0006597222222222222
STEP 1: Edge Detection
620.0
921600
0.0006727430555555556
STEP 1: Edge Detection
626.0
921600
0.0006792534722222223
STEP 1: Edge Detection
626.0
921600
0.0006792534722222223
STEP 1: Edge Detection
620.0
921600
0.0006727430555555556
STEP 1: Edge Detection
624.0
921600
0.0006770833333333334
STEP 1: Edge Detection
624.0
921600
0.0006770833333333334
STEP 1: Edge Detection
622.5
921600
0.0006754557291666667
STEP 1: Edge Detection
622.5
921600
0.0006754557291666667
STEP 1: Edge Detection
622.5
921600
0.0006754557291666667
STEP 1: Edge Detection
STEP 1: Edge Detection
620.0
921600
0.0006727430555555556
STEP 1: Edge Detection
618.0
921600
0.0006705729166666666
STEP 1: Edge Detection
STEP 1: Edge Detection
STEP 1: Edge Detection
STEP 1: Edge Detection
542.0
921600
0.00058810763